# Les dates et les timezone

## De base, nous utilisons des dates *naives*
La librairie standard ne définit pas de timezone. Elle fournit une classe `timezone`, spécialisation simple de `tzinfo` qui s'instancie en passant en paramètre un `timedelta`. Celui-ci doit avoir +/- 12 heures.

In [ ]:
import datetime as dt

print("Maintenant naif  :", dt.datetime.now())

instant = dt.datetime.now(dt.timezone(dt.timedelta(hours=2)))

print("Maintenant aware :", instant)
print("Info timezone    :", instant.tzinfo)

In [ ]:
instant = dt.datetime.now(dt.timezone(dt.timedelta(hours=2), name="France"))
print("Maintenant aware :", instant)
print("Info timezone    :", instant.tzinfo)

In [ ]:
instant = dt.datetime.now(dt.timezone(dt.timedelta(hours=2), name="plus_2"))
autre_instant = dt.datetime.now(dt.timezone(dt.timedelta(hours=1), name="plus_1"))
print("Maintenant aware 2 :", instant)
print("Maintenant aware 1 :", autre_instant)

In [ ]:
autre_instant - instant

## Localisation avec la librairie `zoneinfo`

Python 3.9 a ajouté le [module `zoneinfo`](https://docs.python.org/3/library/zoneinfo.html) qui fournit une implémentation concrète de la base de donnée IANA. Ce module s'appuye sur les données timezone du système si possible. Sur certains systèmes, il peut être nécessaire d'installer un module dédié tel que [`tzdata`](https://tzdata.readthedocs.io/en/latest/).

### Définition d'un timezone réel
Pour illustrer l'usage de `zoneinfo`, nous définissons dans la cellule suivante un timezone pour la France.

In [ ]:
from zoneinfo import ZoneInfo
paris_tz = ZoneInfo('Europe/Paris')
print(paris_tz)
paris_tz

Les valeurs possibles pour les timezone sont fournies par une fonction.

In [ ]:
from zoneinfo import available_timezones
available_timezones()

### Localisation d'une date
Nous pouvons créer une date localisée en ajoutant cette information de timezone :

In [ ]:
print(dt.datetime.now(tz=paris_tz))
dt.datetime.now(tz=paris_tz)

Si nous partons d'une date naive, nous pourrons remplacer le timezone.

In [ ]:
now_naive = dt.datetime.now()
print("Instant actuel naif:", now_naive)

In [ ]:
h_paris_aware = now_naive.replace(tzinfo=paris_tz)
print("France, aware      :", h_paris_aware)

Nous utilisons cette même date pour créer une date *aware* à New York.

In [ ]:
new_york_tz = ZoneInfo('America/New_York')

print("Instant actuel naif:", now_naive)

h_new_york_aware = now_naive.replace(tzinfo=new_york_tz)
print("New York, aware    :", h_new_york_aware)

### Relocalisation d'une date localisée
Nous changeons le timezone de cette date *aware* des US en France.

In [ ]:
h_new_york_in_paris = h_new_york_aware.astimezone(paris_tz)
print("France from US :", h_new_york_in_paris)

Cette nouvelle date n'affiche pas les mêmes heures, mais n'a pas non plus le même timezone. Nous pouvons vérifier qu'il s'agit de la même date.

In [ ]:
h_new_york_aware == h_new_york_in_paris

Les deux dates localisées contiennent la même information d'heure. Nous vérifions qu'il ne s'agit pas du même instant et qu'il y a un décalage d'une heure.

In [ ]:
print("Heure US   :", h_new_york_aware.hour)
print("Heure Fr   :", h_paris_aware.hour)
print("Différence :", h_new_york_aware - h_paris_aware)

### Attention aux méthodes utilisées
Attention à l'usage des méthodes. La localisation d'une date naive se fait bien en remplaçant le timezone. Le décalage (l'usage de `astimezone()`) part du principe que la date naive est locale au système :

In [ ]:
print(now_naive)
print(now_naive.astimezone(paris_tz))
print(now_naive.astimezone(new_york_tz))

### Prise en compte du changement d'heure
Les problèmes de timezone ne se limitent pas au fuseau horaire, mais également à la différence en fonction du changement d'heure. Ci-dessous, nous définissons deux instants à minuit pour un jour en heure d'hiver et un jour en heure d'été.

In [ ]:
winter_day = dt.datetime(2019, 3, 30, tzinfo=paris_tz)
summer_day = dt.datetime(2019, 4, 2, tzinfo=paris_tz)

print("Hiver", winter_day, winter_day.dst(), sep=' | ')
print("Été. ", summer_day, summer_day.dst(), sep=' | ')

In [ ]:
print("Différence :", summer_day - winter_day)
summer_day - winter_day

Pour obtenir la *vraie* différence entre les deux, nous devons passer sur un horaire de référence.

In [ ]:
utc_tz = ZoneInfo('UTC')
summer_day.astimezone(utc_tz) - winter_day.astimezone(utc_tz)

In [ ]:
after_3_days = winter_day + dt.timedelta(days=3)
print("3 jours plus tard :", after_3_days)

In [ ]:
after_3_days = winter_day.astimezone(utc_tz) + dt.timedelta(days=3)
print("3 jours plus tard relocalisé :", after_3_days.astimezone(paris_tz))

## Bonnes pratiques pour manipuler les dates
Voir donc les bonnes pratiques pour manipuler les dates qui doivent être localisées :

In [ ]:
utc_tz = ZoneInfo('UTC')

In [ ]:
meeting = dt.datetime(2020, 10, 15, 15, 30)

In [ ]:
paris_tz = ZoneInfo('Europe/Paris')
meeting = meeting.replace(tzinfo=paris_tz).astimezone(utc_tz)

Ou mieux :

In [ ]:
paris_tz = ZoneInfo('Europe/Paris')
meeting = dt.datetime(2020, 10, 15, 15, 30, tzinfo=paris_tz).astimezone(utc_tz)

In [ ]:
print(meeting)

In [ ]:
new_york_tz = ZoneInfo('America/New_York')
print(f"for New Yorkers : {meeting.astimezone(new_york_tz)}")